In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import MetaTrader5 as mt5

# Set the frequency globally
vbt.settings.array_wrapper['freq'] = '1D'  # Adjust based on your data frequency


def calculate_atr(data, window=14):
    """Calculate Average True Range"""
    high = data['high']
    low = data['low']
    close = data['close'].shift(1)

    tr1 = high - low
    tr2 = abs(high - close)
    tr3 = abs(low - close)

    tr = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
    atr = tr.rolling(window=window).mean()

    return atr


def get_historical_data(symbol, timeframe=mt5.TIMEFRAME_D1, start_date=None,
                        end_date=None):
    """Get historical data from MetaTrader 5"""
    # Initialize MT5 if not already initialized
    if not mt5.initialize():
        print("MT5 initialization failed")
        return None

    # Set default dates if not provided
    if start_date is None:
        start_date = pd.Timestamp.now() - pd.Timedelta(days=1095)  # Last 3 years
    if end_date is None:
        end_date = pd.Timestamp.now()

    # Convert timestamps to MT5 format
    start_date = start_date.to_pydatetime()
    end_date = end_date.to_pydatetime()

    # Request historical data
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)

    if rates is None or len(rates) == 0:
        print(f"Failed to get data for {symbol}")
        return None

    # Convert to pandas DataFrame
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)

    # Rename columns to standard names
    df.rename(columns={'open': 'open', 'high': 'high', 'low': 'low', 'close': 'close',
                       'volume': 'volume'}, inplace=True)

    return df


def optimize_breakout_strategy(data, initial_capital=10000, sl_methods=None,
                               tp_methods=None):
    if sl_methods is None:
        sl_methods = ['atr_based', 'fixed_percent']
    if tp_methods is None:
        tp_methods = ['atr_based', 'fixed_percent']

    # Define parameter ranges to test
    windows = [20, 30, 40, 50]
    std_devs = [1.5, 2.0, 2.5, 3.0]

    results = []

    for window in windows:
        for std_dev in std_devs:
            for sl_method in sl_methods:
                for tp_method in tp_methods:
                    # Create a copy of the data to work with
                    temp_data = data.copy()

                    # Calculate the Bollinger Bands
                    temp_data['sma'] = temp_data['close'].rolling(window=window).mean()
                    temp_data['std'] = temp_data['close'].rolling(window=window).std()
                    temp_data['upper_band'] = temp_data['sma'] + (
                            std_dev * temp_data['std'])
                    temp_data['lower_band'] = temp_data['sma'] - (
                            std_dev * temp_data['std'])

                    # Calculate ATR for stop loss and take profit if needed
                    temp_data['atr'] = calculate_atr(temp_data, window=14)

                    # Generate entry signals for breakout strategy
                    entries = (temp_data['close'] > temp_data['upper_band']) | (
                            temp_data['close'] < temp_data['lower_band'])

                    # Define stop loss and take profit based on selected methods
                    sl_stop = None
                    tp_stop = None

                    if sl_method == 'atr_based':
                        sl_stop = 2.0 * temp_data['atr'] / temp_data['close']
                    elif sl_method == 'fixed_percent':
                        sl_stop = 0.02  # 2% stop loss

                    if tp_method == 'atr_based':
                        tp_stop = 3.0 * temp_data['atr'] / temp_data['close']
                    elif tp_method == 'fixed_percent':
                        tp_stop = 0.03  # 3% take profit

                    # Create portfolio backtest
                    pf = vbt.Portfolio.from_signals(close=temp_data['close'],
                                                    entries=entries, exits=None,
                                                    # We gebruiken SL/TP voor exits
                                                    sl_stop=sl_stop, tp_stop=tp_stop)

                    # Performance metriek verzameling
                    metrics = {'sharpe_ratio': pf.sharpe_ratio(),
                               'total_return': pf.total_return(),
                               'max_drawdown': pf.max_drawdown(),
                               'win_rate': pf.trades.win_rate(), 'window': window,
                               'std_dev': std_dev, 'sl_method': sl_method,
                               'tp_method': tp_method}

                    results.append(metrics)

    if not results:
        return None

    # Convert to DataFrame for easier analysis
    results_df = pd.DataFrame(results)

    # Sort by Sharpe ratio to find optimal parameters
    optimal_params = results_df.sort_values('sharpe_ratio', ascending=False).iloc[0]

    return optimal_params, results_df


# Test both GER40 and US30 indices over a 3-year period
if __name__ == "__main__":
    # Define symbols to test
    symbols = ['GER40.cash', 'US30.cash']

    # Store results for comparison
    all_results = {}
    all_detailed_results = {}

    # Get start and end dates for consistent backtest period
    end_date = pd.Timestamp.now()
    start_date = end_date - pd.Timedelta(days=1095)  # 3 years

    for symbol in symbols:
        print(f"\nTesting strategy on {symbol}...")
        data = get_historical_data(symbol, start_date=start_date, end_date=end_date)

        if data is not None:
            print(f"Data period: {data.index.min()} to {data.index.max()}")
            print(f"Total days: {len(data)}")

            optimization_results, detailed_results = optimize_breakout_strategy(data)
            all_results[symbol] = optimization_results
            all_detailed_results[symbol] = detailed_results

            print(f"Optimization results for {symbol}:")
            print(optimization_results)

            # Save top 5 strategies for this symbol
            top_strategies = detailed_results.sort_values('sharpe_ratio',
                                                          ascending=False).head(5)
            print(f"\nTop 5 strategies for {symbol}:")
            print(top_strategies)
        else:
            print(f"Failed to get data for {symbol}, cannot proceed with optimization.")

    # Compare results
    if len(all_results) > 0:
        print("\nComparison of optimal strategies:")
        comparison_df = pd.DataFrame(all_results)
        print(comparison_df)

        # Create a detailed performance report
        print("\n============ DETAILED PERFORMANCE REPORT ============")
        for symbol, optimal_params in all_results.items():
            print(f"\n{symbol} - Optimal Strategy Performance:")
            print(f"Window: {optimal_params['window']}")
            print(f"Standard Deviation: {optimal_params['std_dev']}")
            print(f"Stop Loss Method: {optimal_params['sl_method']}")
            print(f"Take Profit Method: {optimal_params['tp_method']}")
            print(f"Sharpe Ratio: {optimal_params['sharpe_ratio']:.4f}")
            print(f"Total Return: {optimal_params['total_return'] * 100:.2f}%")
            print(f"Maximum Drawdown: {optimal_params['max_drawdown'] * 100:.2f}%")
            print(f"Win Rate: {optimal_params['win_rate'] * 100:.2f}%")


Testing strategy on GER40.cash...
Data period: 2022-04-11 00:00:00 to 2025-04-04 00:00:00
Total days: 763
Optimization results for GER40.cash:
sharpe_ratio         2.047109
total_return         0.657011
max_drawdown        -0.068635
win_rate                 0.64
window                     50
std_dev                   2.0
sl_method       fixed_percent
tp_method           atr_based
Name: 54, dtype: object

Top 5 strategies for GER40.cash:
    sharpe_ratio  total_return  max_drawdown  win_rate  window  std_dev  \
54      2.047109      0.657011     -0.068635  0.640000      50      2.0   
12      1.803277      0.180092     -0.022858  0.800000      20      3.0   
40      1.767145      0.387487     -0.081506  0.727273      40      2.5   
28      1.728608      0.209510     -0.030342  0.800000      30      3.0   
53      1.710347      0.513693     -0.079493  0.708333      50      2.0   

        sl_method      tp_method  
54  fixed_percent      atr_based  
12      atr_based      atr_based  
40

In [2]:
import datetime
import pytz

# Definieer tijdzones
belgian_tz = pytz.timezone('Europe/Brussels')
utc_tz = pytz.UTC

# Huidige tijd in verschillende tijdzones
now = datetime.datetime.now()
belgian_time = belgian_tz.localize(now)
utc_time = now.astimezone(utc_tz)

print(f"Huidige tijd in België: {belgian_time}")
print(f"Huidige tijd in UTC: {utc_time}")
print(f"Tijdsverschil: {belgian_time.utcoffset()}")

Huidige tijd in België: 2025-04-07 07:17:27.672409+02:00
Huidige tijd in UTC: 2025-04-07 05:17:27.672409+00:00
Tijdsverschil: 2:00:00
